In [2]:
import pandas as pd
import bs4
import requests
import json


/tmp/ipykernel_14901/2595686090.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [41]:
# single car example

url = 'https://evkx.net/models/tesla/model_y/model_y_long_range/chargingcurve/'

tables = pd.read_html(url)

CHARGE_TIMES_INDEX = 1

charge_time_table = tables[CHARGE_TIMES_INDEX]

In [42]:
model_y_charge_str_times = charge_time_table['Time'].tolist()



In [44]:
def hms_to_seconds(hms):

    hours = 0

    split_hours = hms.split('h')
    hours_index = 0
    m_s_index = 1


    # handle case where theres no hours
    if 'h' in hms:
        hours = int(split_hours[hours_index])
    else:
        m_s_index = 0
    
    split_minutes = split_hours[m_s_index].split('m')
    minutes_index = 0
    seconds_index = 1

    minutes = int(split_minutes[minutes_index])

    split_seconds = split_minutes[seconds_index].split('s')
    seconds_index = 0

    seconds = int(split_seconds[seconds_index])

    total_seconds = 60 * 60 * hours + 60 * minutes + seconds

    return total_seconds

hms_to_seconds('1 m 1 s')

61

In [46]:
def get_charge_times(url):
    tables = pd.read_html(url)
    
    CHARGE_TIMES_INDEX = 1
    charge_time_table = tables[CHARGE_TIMES_INDEX]

    hms_string_times = charge_time_table['Time'].tolist()

    return [hms_to_seconds(i) for i in hms_string_times]

get_charge_times('https://evkx.net/models/acura/zdx/zdx_type_s/chargingcurve/')

[0,
 70,
 129,
 181,
 226,
 267,
 304,
 337,
 368,
 397,
 424,
 449,
 474,
 499,
 524,
 548,
 571,
 595,
 618,
 641,
 663,
 685,
 707,
 729,
 750,
 771,
 792,
 813,
 834,
 855,
 876,
 898,
 919,
 940,
 962,
 984,
 1006,
 1028,
 1052,
 1075,
 1100,
 1124,
 1150,
 1176,
 1203,
 1230,
 1257,
 1285,
 1313,
 1341,
 1370,
 1399,
 1428,
 1457,
 1486,
 1515,
 1544,
 1573,
 1602,
 1631,
 1660,
 1690,
 1719,
 1748,
 1778,
 1808,
 1838,
 1869,
 1901,
 1933,
 1965,
 1998,
 2031,
 2065,
 2100,
 2135,
 2171,
 2207,
 2244,
 2282,
 2320,
 2374,
 2464,
 2559,
 2660,
 2768,
 2883,
 3007,
 3140,
 3285,
 3444,
 3620,
 3816,
 4039,
 4296,
 4599,
 4970,
 5447,
 6115,
 7227,
 10566]

In [9]:
model_y_charge_seconds = [hms_to_seconds(i) for i in model_y_charge_str_times]

In [52]:
makers_url = 'https://evkx.net/models/'
res = requests.get(makers_url)
res.raise_for_status()
makes_soup = bs4.BeautifulSoup(res.text, 'html.parser')

# get the car maker cards
maker_cards = makes_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')

cars = {}
for card in maker_cards[0]:
    # extract the model name
    print(card)
    maker_name = card.select('h2')[0].get_text()
    # extract the link
    link = card.select('a')[0].get('href')

    # follow link to get the models
    models_url = makers_url + link
    models_resp = requests.get(models_url)
    models_resp.raise_for_status()
    models_soup = bs4.BeautifulSoup(models_resp.text, 'html.parser')

    model_cards = models_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')

    models = {}
    for model in model_cards:
        model_name = model.select('h3')[0].get_text()
        link = model.select('a')[0].get('href')
        
        sub_model_url = models_url + link
        sub_models_resp = requests.get(sub_model_url)
        sub_models_resp.raise_for_status()
        sub_models_soup = bs4.BeautifulSoup(sub_models_resp.text, 'html.parser')

        sub_model_cards = sub_models_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')
        sub_models = {}

        for sub_model in sub_model_cards:
            sub_model_name = sub_model.select('h3')[0].get_text()
            charge_curve_selector = 'li:nth-child(3) > a:nth-child(1)'
            link = sub_model.select(charge_curve_selector)[0].get('href')
            charge_curve_url = sub_model_url + link

            charge_times = get_charge_times(charge_curve_url)
            sub_models[sub_model_name] = charge_times

        models[model_name] = sub_models
    cars[maker_name] = models
    

print(cars)

<a href="acura/"><img alt="Acura" class="img-fluid mb-2" src="https://media.evkx.net/multimedia/models/acura/brandevs_st.jpg"/></a>


IndexError: list index out of range

[{'name': 'Acura', 'link': 'acura/', 'models': [{'name': 'Acura ZDX', 'link': 'zdx'}]}, {'name': 'Aion', 'link': 'aion/', 'models': [{'name': 'Aion Hyper GT', 'link': 'hyper_gt'}]}, {'name': 'Audi', 'link': 'audi/'}, {'name': 'BMW', 'link': 'bmw/'}, {'name': 'BYD', 'link': 'byd/'}, {'name': 'Cadillac', 'link': 'cadillac/'}, {'name': 'Chevrolet', 'link': 'chevrolet/'}, {'name': 'Citroën', 'link': 'citroën/'}, {'name': 'CUPRA', 'link': 'cupra/'}, {'name': 'DS', 'link': 'ds/'}, {'name': 'Fiat', 'link': 'fiat/'}, {'name': 'Fisker', 'link': 'fisker/'}, {'name': 'Ford', 'link': 'ford/'}, {'name': 'Genesis', 'link': 'genesis/'}, {'name': 'GMC', 'link': 'gmc/'}, {'name': 'HiPhi', 'link': 'hiphi/'}, {'name': 'Honda', 'link': 'honda/'}, {'name': 'Hongqi', 'link': 'hongqi/'}, {'name': 'Hyundai', 'link': 'hyundai/'}, {'name': 'Jaguar', 'link': 'jaguar/'}, {'name': 'Jeep', 'link': 'jeep/'}, {'name': 'Kia', 'link': 'kia/'}, {'name': 'Lexus', 'link': 'lexus/'}, {'name': 'Lotus', 'link': 'lotus/'}, {'